# Code Archive NLP ICA

In [ ]:
##### Only using praw to gather the latest 1000 posts
for submission in reddit.subreddit('ethereum').new(limit=None):
    raw_data = raw_data.append({
        "title":submission.title,
        "text":submission.selftext,
        "score":submission.score,
        "time":pd.to_datetime(submission.created, unit="s"),
        "upvote_ratio":submission.upvote_ratio
    }, ignore_index=True)

In [ ]:
# Tokenize
tokenizer = RegexpTokenizer(r'\w+')
stop_words = stopwords.words('english')

# Preprocess
tokens = []
for text in raw_data["text"]:
    text = text.lower()
    text = ''.join(ch for ch in text if ch not in string.punctuation)
    text = tokenizer.tokenize(text)
    text = [t for t in text if t not in stop_words]
    tokens.append(text)

raw_data["text"] = tokens

tokens = []
for text in raw_data["title"]:
    text = text.lower()
    text = ''.join(ch for ch in text if ch not in string.punctuation)
    text = tokenizer.tokenize(text)
    text = [t for t in text if t not in stop_words]
    tokens.append(text)

raw_data["title"] = tokens

In [ ]:
# Use requests package to gather data!

import requests

#Authentication
auth = requests.auth.HTTPBasicAuth("kY_y2GGoj9wIVxupLMPbJg", "7tTfZujRWd_oVqE0CZJWntY7IzcKWw")
#Login
data = {"grant_type": "password",
        "username": "Boegun", #Make this an input
        "password": "Bezaza12"} #Make this an input
headers = {"User-Agent": "ICATopic/0.0.1"}
res = requests.post("https://www.reddit.com/api/v1/access_token",
                    auth=auth, data=data, headers=headers)
TOKEN = res.json()["access_token"]
headers = {**headers, **{"Authorization": f"bearer {TOKEN}"}}
requests.get("https://oautch.reddit.com/api/v1/me", headers=headers)

#Request data, Pulling data from subbreddit "ethereum" with search query "the merge"
res = requests.get("https://oauth.reddit.com/r/ethereum/search/?q=the%20merge&restrict_sr=1&sr_nsfw=", headers=headers)

#Build a dataframe for analysis
df = pd.DataFrame() #initialize dataframe
for post in res.json()["data"]["children"]:
    #append relevant attributes to dataframe
    df = df.append({
        #"subreddit": post["data"]["subreddit"],
        "title": post["data"]["title"],
        "selftext": post["data"]["selftext"],
        "upvote_ratio": post["data"]["upvote_ratio"],
        #"ups": post["data"]["ups"], #Always equal score??
        #"downs": post["data"]["downs"], #Always 0???
        "score": post["data"]["score"],
        "time":post["data"]["created"]
    },
    ignore_index=True
    )

In [ ]:
#raw_data = raw_data[~raw_data['text'].isin(['[removed]', '[deleted]']).dropna()]